In [1]:
import os
cuda_path = 'C:\\Program Files\\NVIDIA GPU Computing Toolkit\\CUDA\\v10.0\\bin'
cudnn_path = 'D:\\Work\\software\\cuda\\bin'
os.environ["PATH"] += os.pathsep + cuda_path
os.environ["PATH"] += os.pathsep + cudnn_path
import keras
import tensorflow as tf
from keras.backend.tensorflow_backend import set_session
config = tf.ConfigProto()
config.gpu_options.per_process_gpu_memory_fraction = 0.9
config.gpu_options.allow_growth = True  # dynamically grow the memory used on the GPU
sess = tf.Session(config=config)
set_session(sess)

Using TensorFlow backend.
C:\Users\Richard\Anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
C:\Users\Richard\Anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
C:\Users\Richard\Anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
C:\Users\Richard\Anaconda3\lib\site-packages\tensorflow\python\framework\d

In [2]:
#import analysis
import sys
sys.path.insert(0,'../scripts/')

import test_features
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib.pyplot import subplots,scatter
import seaborn as sns
from sklearn.externals import joblib
from sklearn.cluster import KMeans
from sklearn.metrics import adjusted_rand_score, adjusted_mutual_info_score
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix 
from sklearn.model_selection import cross_val_score
import os
#import analysis
def read_data(file_dir,DB_file_dir):
    
    data = []
    label = []
    
    print('loading..{}'.format(file_dir))
    df=pd.read_pickle(file_dir)
    psm_ID=list(pd.read_csv(DB_file_dir)['Scan'])
    y=np.zeros(df.shape[0])

    for i in range(0,df.shape[0]):
        if df.index[i] in psm_ID:
            y[i]=1

    X = df.drop('RT',axis=1).fillna(0).values
    y =y.astype(int)[X.sum(axis=1)!=0]
    X = X[X.sum(axis=1)!=0]
    X = X/X.sum(axis=1).reshape(X.shape[0],1) #normalization
    data.append(X)
    label.append(y)
    del X

    data = np.concatenate((data))
    label = np.concatenate((label))
    return data,label

from numpy.random import randint,seed,choice
def under_sampler(data,label,n_sample):
    
    index_maj = np.where(label==0)[0]
    index_min = np.where(label==1)[0]
    
    seed(19)
    sample_index = choice(len(index_maj),size=n_sample,replace=False)
    index_maj_sampled = index_maj[sample_index]
    data_sampled = data[np.concatenate((index_maj_sampled,index_min))]
    label_sampled = label[np.concatenate((index_maj_sampled,index_min))]
    
    shuffle_index = choice(len(data_sampled),size=len(data_sampled),replace=False)
    data_sampled = data_sampled[shuffle_index]
    label_sampled = label_sampled[shuffle_index]
    
    return data_sampled,label_sampled

def load_sample_data(file_dir,DB_file_dir,n_sample):
    
    X_train,X_test,Y_train,Y_test = [],[],[],[]
    for i in range(len(file_dir)):
        data,label = read_data(file_dir[i],DB_file_dir[i])
        x_train, x_test, y_train, y_test = train_test_split(data, label, test_size=0.3, random_state=1)
        del data
        
        x_train_sampled,y_train_sampled = under_sampler(x_train,y_train,len(y_train[y_train==1])*n_sample)
        del x_train,y_train
        X_train.append(x_train_sampled)
        X_test.append(x_test)
        Y_train.append(y_train_sampled)
        Y_test.append(y_test)
        del x_test,y_test,x_train_sampled,y_train_sampled
        
    X_train = np.concatenate(X_train)
    X_test = np.concatenate(X_test)
    Y_train = np.concatenate(Y_train)
    Y_test = np.concatenate(Y_test)

    return X_train,X_test,Y_train,Y_test

C:\Users\Richard\Anaconda3\lib\site-packages\sklearn\externals\joblib\__init__.py:15: DeprecationWarning: sklearn.externals.joblib is deprecated in 0.21 and will be removed in 0.23. Please import this functionality directly from joblib, which can be installed with: pip install joblib. If this warning is raised when loading pickled models, you may need to re-serialize those models with scikit-learn 0.21+.
  warnings.warn(msg, category=DeprecationWarning)


In [3]:
file_dir = ['../Orbitrap_aplysia/abd.pkl','../Orbitrap_aplysia/buc.pkl','../Orbitrap_aplysia/cer.pkl']
DB_file_dir = ['../Aplysia_ganglia/Abdominal/DB search psm.csv','../Aplysia_ganglia/Buccal/DB search psm.csv',
              '../Aplysia_ganglia/Cerebral/DB search psm.csv']

X_train,X_test,Y_train,Y_test = load_sample_data(file_dir,DB_file_dir,2)

loading..../Orbitrap_aplysia/abd.pkl
loading..../Orbitrap_aplysia/buc.pkl
loading..../Orbitrap_aplysia/cer.pkl


In [4]:
X_train = X_train.reshape((X_train.shape[0],X_train.shape[1],1))
X_test = X_test.reshape((X_test.shape[0],X_test.shape[1],1))

In [9]:
from __future__ import print_function
from keras.layers import Dense, Conv1D, Conv2D, BatchNormalization, Activation
from keras.layers import AveragePooling1D, AveragePooling2D, Input, Flatten
from keras.optimizers import Adam
from keras.callbacks import ModelCheckpoint, LearningRateScheduler
from keras.callbacks import ReduceLROnPlateau
from keras.preprocessing.image import ImageDataGenerator
from keras.regularizers import l2
from keras import backend as K
from keras.models import Model
from keras.datasets import cifar10
import numpy as np
import os

# from keras.backend.tensorflow_backend import set_session
# import tensorflow as tf
# K.clear_session()
# config = tf.ConfigProto()
# config.gpu_options.allow_growth = True  # dynamically grow the memory used on the GPU
# config.log_device_placement = True  # to log device placement (on which device the operation ran)
# sess = tf.Session(config=config)
# set_session(sess)  # set this TensorFlow session as the default session for Keras


# Training parameters
batch_size = 32  # orig paper trained all networks with batch_size=128
epochs = 100
data_augmentation = False
num_classes = 2
input_shape = X_train.shape[1:]
scale_weight = Y_train[Y_train==0].shape[0]/Y_train[Y_train==1].shape[0]

n = 1
version = 2
# Computed depth from supplied model parameter n
if version == 1:
    depth = n * 6 + 2
elif version == 2:
    depth = n * 9 + 2

# Model name, depth and version
model_type = 'ResNet%dv%d' % (depth, version)

# Input image dimensions.
# Convert class vectors to binary class matrices.
#y_train_aug = keras.utils.to_categorical(y_train_aug, num_classes)
Y_train_ = keras.utils.to_categorical(Y_train, num_classes)
Y_test_ = keras.utils.to_categorical(Y_test, num_classes)

In [10]:
def lr_schedule(epoch):
    """Learning Rate Schedule

    Learning rate is scheduled to be reduced after 80, 120, 160, 180 epochs.
    Called automatically every epoch as part of callbacks during training.

    # Arguments
        epoch (int): The number of epochs

    # Returns
        lr (float32): learning rate
    """
    lr = 1e-2
    if epoch > 100:
        lr *= 0.5e-6
    elif epoch > 80:
        lr *= 1e-6
    elif epoch > 60:
        lr *= 1e-5
    elif epoch > 40:
        lr *= 1e-4
    print('Learning rate: ', lr)
    return lr


def resnet_layer(inputs,
                 num_filters=8,
                 kernel_size=3,
                 strides=1,
                 activation='relu',
                 batch_normalization=True,
                 conv_first=True):
    """2D Convolution-Batch Normalization-Activation stack builder

    # Arguments
        inputs (tensor): input tensor from input image or previous layer
        num_filters (int): Conv2D number of filters
        kernel_size (int): Conv2D square kernel dimensions
        strides (int): Conv2D square stride dimensions
        activation (string): activation name
        batch_normalization (bool): whether to include batch normalization
        conv_first (bool): conv-bn-activation (True) or
            bn-activation-conv (False)

    # Returns
        x (tensor): tensor as input to the next layer
    """
    conv = Conv1D(num_filters,
                  kernel_size=kernel_size,
                  strides=strides,
                  padding='same',
                  kernel_initializer='he_normal',
                  kernel_regularizer=l2(1e-4))

    x = inputs
    if conv_first:
        x = conv(x)
        if batch_normalization:
            x = BatchNormalization()(x)
        if activation is not None:
            x = Activation(activation)(x)
    else:
        if batch_normalization:
            x = BatchNormalization()(x)
        if activation is not None:
            x = Activation(activation)(x)
        x = conv(x)
    return x

def resnet_v2(input_shape, depth, num_classes=2):
    """ResNet Version 2 Model builder [b]

    Stacks of (1 x 1)-(3 x 3)-(1 x 1) BN-ReLU-Conv2D or also known as
    bottleneck layer
    First shortcut connection per layer is 1 x 1 Conv2D.
    Second and onwards shortcut connection is identity.
    At the beginning of each stage, the feature map size is halved (downsampled)
    by a convolutional layer with strides=2, while the number of filter maps is
    doubled. Within each stage, the layers have the same number filters and the
    same filter map sizes.


    # Arguments
        input_shape (tensor): shape of input image tensor
        depth (int): number of core convolutional layers
        num_classes (int): number of classes (CIFAR10 has 10)

    # Returns
        model (Model): Keras model instance
    """
    if (depth - 2) % 9 != 0:
        raise ValueError('depth should be 9n+2 (eg 56 or 110 in [b])')
    # Start model definition.
    num_filters_in = 16
    num_res_blocks = int((depth - 2) / 9)

    inputs = Input(shape=input_shape)
    # v2 performs Conv1D with BN-ReLU on input before splitting into 2 paths
    x = resnet_layer(inputs=inputs,
                     num_filters=num_filters_in,
                     conv_first=True)

    # Instantiate the stack of residual units
    for stage in range(3):
        for res_block in range(num_res_blocks):
            activation = 'relu'
            batch_normalization = True
            strides = 1
            if stage == 0:
                num_filters_out = num_filters_in * 4
                if res_block == 0:  # first layer and first stage
                    activation = None
                    batch_normalization = False
            else:
                num_filters_out = num_filters_in * 2
                if res_block == 0:  # first layer but not first stage
                    strides = 2    # downsample

            # bottleneck residual unit
            y = resnet_layer(inputs=x,
                             num_filters=num_filters_in,
                             kernel_size=1,
                             strides=strides,
                             activation=activation,
                             batch_normalization=batch_normalization,
                             conv_first=False)
            y = resnet_layer(inputs=y,
                             num_filters=num_filters_in,
                             conv_first=False)
            y = resnet_layer(inputs=y,
                             num_filters=num_filters_out,
                             kernel_size=1,
                             conv_first=False)
            if res_block == 0:
                # linear projection residual shortcut connection to match
                # changed dims
                x = resnet_layer(inputs=x,
                                 num_filters=num_filters_out,
                                 kernel_size=1,
                                 strides=strides,
                                 activation=None,
                                 batch_normalization=False)
            x = keras.layers.add([x, y])

        num_filters_in = num_filters_out

    # Add classifier on top.
    # v2 has BN-ReLU before Pooling
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
    x = AveragePooling1D(pool_size=8)(x)
    y = Flatten()(x)
    outputs = Dense(num_classes,
                    activation='softmax',
                    kernel_initializer='he_normal')(y)

    # Instantiate model.
    model = Model(inputs=inputs, outputs=outputs)
    return model

In [7]:
from keras import backend as K

def recall_m(y_true, y_pred):
        true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
        possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
        recall = true_positives / (possible_positives + K.epsilon())
        return recall

def precision_m(y_true, y_pred):
        true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
        predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
        precision = true_positives / (predicted_positives + K.epsilon())
        return precision

def f1_m(y_true, y_pred):
    precision = precision_m(y_true, y_pred)
    recall = recall_m(y_true, y_pred)
    return 2*((precision*recall)/(precision+recall+K.epsilon()))

INTERESTING_CLASS_ID =  1 # Choose the class of interest

def single_class_accuracy(y_true, y_pred):
    class_id_true = K.argmax(y_true, axis=-1)
    class_id_preds = K.argmax(y_pred, axis=-1)
    accuracy_mask = K.cast(K.equal(class_id_preds, INTERESTING_CLASS_ID), 'int32')
    #recall_mask = K.cast(K.equal(class_id_true, INTERESTING_CLASS_ID), 'int32')

    class_acc_tensor = K.cast(K.equal(class_id_true, class_id_preds), 'int32') * accuracy_mask
    #class_recall_tensor = K.cast(K.equal(class_id_true, class_id_true), 'int32') * recall_mask
    class_acc = K.sum(class_acc_tensor) / K.maximum(K.sum(accuracy_mask), 1)
    #class_recall = K.sum(class_recall_tensor) / K.maximum(K.sum(recall_mask), 1)
    return class_acc

def single_class_recall(y_true, y_pred):
    class_id_true = K.argmax(y_true, axis=-1)
    class_id_preds = K.argmax(y_pred, axis=-1)
    #accuracy_mask = K.cast(K.equal(class_id_preds, INTERESTING_CLASS_ID), 'int32')
    recall_mask = K.cast(K.equal(class_id_true, INTERESTING_CLASS_ID), 'int32')

    #class_acc_tensor = K.cast(K.equal(class_id_true, class_id_preds), 'int32') * accuracy_mask
    class_recall_tensor = K.cast(K.equal(class_id_true, class_id_preds), 'int32') * recall_mask
    #class_acc = K.sum(class_acc_tensor) / K.maximum(K.sum(accuracy_mask), 1)
    class_recall = K.sum(class_recall_tensor) / K.maximum(K.sum(recall_mask), 1)
    return class_recall

if version == 2:
    model = resnet_v2(input_shape=input_shape, depth=depth)
else:
    model = resnet_v1(input_shape=input_shape, depth=depth)

model.compile(loss='categorical_crossentropy', optimizer=keras.optimizers.Adadelta(lr=1e-3),
                  metrics=[recall_m,precision_m,f1_m,single_class_accuracy,single_class_recall])
model.summary()
print(model_type)

W0115 17:05:13.616310 18648 deprecation_wrapper.py:119] From C:\Users\Richard\Anaconda3\lib\site-packages\keras\backend\tensorflow_backend.py:74: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.

W0115 17:05:13.645348 18648 deprecation_wrapper.py:119] From C:\Users\Richard\Anaconda3\lib\site-packages\keras\backend\tensorflow_backend.py:517: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.

W0115 17:05:13.829168 18648 deprecation_wrapper.py:119] From C:\Users\Richard\Anaconda3\lib\site-packages\keras\backend\tensorflow_backend.py:4185: The name tf.truncated_normal is deprecated. Please use tf.random.truncated_normal instead.

W0115 17:05:14.158424 18648 deprecation_wrapper.py:119] From C:\Users\Richard\Anaconda3\lib\site-packages\keras\backend\tensorflow_backend.py:133: The name tf.placeholder_with_default is deprecated. Please use tf.compat.v1.placeholder_with_default instead.

W0115 17:05:15.478189 18648

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 12001, 1)     0                                            
__________________________________________________________________________________________________
conv1d_1 (Conv1D)               (None, 12001, 16)    64          input_1[0][0]                    
__________________________________________________________________________________________________
batch_normalization_1 (BatchNor (None, 12001, 16)    64          conv1d_1[0][0]                   
__________________________________________________________________________________________________
activation_1 (Activation)       (None, 12001, 16)    0           batch_normalization_1[0][0]      
__________________________________________________________________________________________________
conv1d_2 (

In [11]:
save_dir = os.path.join(os.getcwd(), 'models')
model_name = 'Sampled_%s_model.{epoch:03d}.h5' % model_type
if not os.path.isdir(save_dir):
    os.makedirs(save_dir)
filepath = os.path.join(save_dir, model_name)

# Prepare callbacks for model saving and for learning rate adjustment.
checkpoint = ModelCheckpoint(filepath=filepath,
                             monitor='val_single_class_accuracy',
                             verbose=1,
                             save_best_only=True)

lr_scheduler = LearningRateScheduler(lr_schedule)

lr_reducer =  ReduceLROnPlateau(min_lr=1e-6, monitor='val_single_class_accuracy', verbose=1,factor=0.8,patience=20)

callbacks = [checkpoint, lr_reducer, lr_scheduler]

# Run training, with or without data augmentation.
if not data_augmentation:
    print('Not using data augmentation.')
    model.fit(X_train, Y_train_,
              batch_size=batch_size,
              epochs=epochs,
              validation_data=(X_test, Y_test_),
              shuffle=True,
              callbacks=callbacks,class_weight={0:1,1:scale_weight})

W0115 17:05:45.808696 18648 deprecation.py:323] From C:\Users\Richard\Anaconda3\lib\site-packages\tensorflow\python\ops\math_grad.py:1250: add_dispatch_support.<locals>.wrapper (from tensorflow.python.ops.array_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


Not using data augmentation.
Train on 18006 samples, validate on 41590 samples
Epoch 1/100
Learning rate:  0.01
18006/18006 [==============================] - 151s 8ms/step - loss: 1.0233 - recall_m: 0.6734 - precision_m: 0.6734 - f1_m: 0.6734 - single_class_accuracy: 0.5039 - single_class_recall: 0.7587 - val_loss: 0.7583 - val_recall_m: 0.7335 - val_precision_m: 0.7335 - val_f1_m: 0.7335 - val_single_class_accuracy: 0.1713 - val_single_class_recall: 0.7025

Epoch 00001: val_single_class_accuracy improved from -inf to 0.17128, saving model to D:\Work\projects\DL for MS\Notebooks\models\Sampled_ResNet11v2_model.001.h5
Epoch 2/100
Learning rate:  0.01
18006/18006 [==============================] - 141s 8ms/step - loss: 0.8333 - recall_m: 0.8101 - precision_m: 0.8101 - f1_m: 0.8101 - single_class_accuracy: 0.6711 - single_class_recall: 0.8520 - val_loss: 0.6843 - val_recall_m: 0.8105 - val_precision_m: 0.8105 - val_f1_m: 0.8105 - val_single_class_accuracy: 0.2292 - val_single_class_recal

18006/18006 [==============================] - 152s 8ms/step - loss: 0.6123 - recall_m: 0.8984 - precision_m: 0.8984 - f1_m: 0.8984 - single_class_accuracy: 0.8050 - single_class_recall: 0.9210 - val_loss: 0.5927 - val_recall_m: 0.8691 - val_precision_m: 0.8691 - val_f1_m: 0.8691 - val_single_class_accuracy: 0.3109 - val_single_class_recall: 0.7567

Epoch 00017: val_single_class_accuracy did not improve from 0.31237
Epoch 18/100
Learning rate:  0.01
18006/18006 [==============================] - 152s 8ms/step - loss: 0.6083 - recall_m: 0.8975 - precision_m: 0.8975 - f1_m: 0.8975 - single_class_accuracy: 0.8021 - single_class_recall: 0.9192 - val_loss: 0.6108 - val_recall_m: 0.8618 - val_precision_m: 0.8618 - val_f1_m: 0.8618 - val_single_class_accuracy: 0.3012 - val_single_class_recall: 0.7659

Epoch 00018: val_single_class_accuracy did not improve from 0.31237
Epoch 19/100
Learning rate:  0.01
18006/18006 [==============================] - 152s 8ms/step - loss: 0.6015 - recall_m: 0.90


Epoch 00034: val_single_class_accuracy did not improve from 0.36722
Epoch 35/100
Learning rate:  0.01
18006/18006 [==============================] - 145s 8ms/step - loss: 0.5353 - recall_m: 0.9191 - precision_m: 0.9191 - f1_m: 0.9191 - single_class_accuracy: 0.8397 - single_class_recall: 0.9389 - val_loss: 0.5755 - val_recall_m: 0.8752 - val_precision_m: 0.8752 - val_f1_m: 0.8752 - val_single_class_accuracy: 0.3236 - val_single_class_recall: 0.7633

Epoch 00035: val_single_class_accuracy did not improve from 0.36722
Epoch 36/100
Learning rate:  0.01
18006/18006 [==============================] - 144s 8ms/step - loss: 0.5329 - recall_m: 0.9212 - precision_m: 0.9212 - f1_m: 0.9212 - single_class_accuracy: 0.8418 - single_class_recall: 0.9416 - val_loss: 0.5252 - val_recall_m: 0.8945 - val_precision_m: 0.8945 - val_f1_m: 0.8945 - val_single_class_accuracy: 0.3541 - val_single_class_recall: 0.7330

Epoch 00036: val_single_class_accuracy did not improve from 0.36722
Epoch 37/100
Learning r

KeyboardInterrupt: 

In [34]:
model = keras.models.load_model('models/ICC_ResNet20v2_model.001.h5')

In [ ]:
y_pred_ = y_pred.argmax(axis=1)
classification_report(y_test, y_pred, output_dict=True)

In [49]:
import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv1D, Reshape#, MaxPooling1D
from keras.callbacks import ReduceLROnPlateau
from keras.layers.noise import GaussianNoise
#Hyperparameters for the network
DENSE = 256
DROPOUT = 0.5
C1_K  = 8 #Number of kernels/feature extractors for first layer
C1_S  = 3 #Width of the convolutional mini networks
C2_K  = 8
C2_S  = 3
C3_K  = 4
C3_S  = 3
C4_K  = 1
C4_S  = 3

activation='relu'

input_dim = x_train.shape[1]

#The model
def make_model():
    model = Sequential()
    #Adding a bit of GaussianNoise also works as regularization
    model.add(GaussianNoise(0, input_shape=(input_dim,)))
    #First two is number of filter + kernel size
    model.add(Reshape((input_dim, 1)))
    model.add(Conv1D(C1_K, (C1_S), activation=activation, border_mode="same"))
    model.add(Conv1D(C2_K, (C2_S), border_mode="same", activation=activation))
    model.add(Conv1D(C3_K, (C3_S), border_mode="same", activation=activation))
    model.add(Conv1D(C4_K, (C4_S), border_mode="same", activation=activation))
    model.add(Flatten())
    model.add(Dropout(DROPOUT))
    model.add(Dense(DENSE, activation=activation))
    model.add(Dense(2, activation='softmax'))

    model.compile(loss='binary_crossentropy', optimizer=keras.optimizers.Adadelta(lr=1e-3),
                  metrics=[recall_m,precision_m,f1_m,single_class_accuracy,single_class_recall])#, metrics=['mean_absolute_error'])

    return model

In [50]:
#keras.backend.clear_session()
model = make_model()
print(model.summary())

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
gaussian_noise_15 (GaussianN (None, 12001)             0         
_________________________________________________________________
reshape_15 (Reshape)         (None, 12001, 1)          0         
_________________________________________________________________
conv1d_45 (Conv1D)           (None, 12001, 8)          32        
_________________________________________________________________
conv1d_46 (Conv1D)           (None, 12001, 8)          200       
_________________________________________________________________
conv1d_47 (Conv1D)           (None, 12001, 4)          100       
_________________________________________________________________
conv1d_48 (Conv1D)           (None, 12001, 1)          13        
_________________________________________________________________
flatten_15 (Flatten)         (None, 12001)             0         
__________

C:\Users\Richard\Anaconda3\lib\site-packages\ipykernel_launcher.py:30: UserWarning: Update your `Conv1D` call to the Keras 2 API: `Conv1D(8, 3, activation="relu", padding="same")`
C:\Users\Richard\Anaconda3\lib\site-packages\ipykernel_launcher.py:31: UserWarning: Update your `Conv1D` call to the Keras 2 API: `Conv1D(8, 3, activation="relu", padding="same")`
C:\Users\Richard\Anaconda3\lib\site-packages\ipykernel_launcher.py:32: UserWarning: Update your `Conv1D` call to the Keras 2 API: `Conv1D(4, 3, activation="relu", padding="same")`
C:\Users\Richard\Anaconda3\lib\site-packages\ipykernel_launcher.py:33: UserWarning: Update your `Conv1D` call to the Keras 2 API: `Conv1D(1, 3, activation="relu", padding="same")`


In [51]:
rdlr = ReduceLROnPlateau( min_lr=1e-6, monitor='val_single_class_accuracy', verbose=1,factor=0.8,patience=20)
save_dir = os.path.join(os.getcwd(), 'models')
model_name = 'abd_CNN_model.{epoch:03d}.h5' 
if not os.path.isdir(save_dir):
    os.makedirs(save_dir)
filepath = os.path.join(save_dir, model_name)

# Prepare callbacks for model saving and for learning rate adjustment.
checkpoint = ModelCheckpoint(filepath=filepath,
                             monitor='val_single_class_accuracy',
                             verbose=1,
                             save_best_only=True)

model.fit(x_train, y_train, epochs=200, batch_size=128, validation_data=(x_test, y_test), callbacks=[rdlr,checkpoint],
         class_weight={0:1,1:scale_weight})

Train on 65244 samples, validate on 27962 samples
Epoch 1/200
65244/65244 [==============================] - 21s 321us/step - loss: 1.3127 - recall_m: 0.3555 - precision_m: 0.3555 - f1_m: 0.3555 - single_class_accuracy: 0.0541 - single_class_recall: 0.6721 - val_loss: 0.6935 - val_recall_m: 0.4027 - val_precision_m: 0.4027 - val_f1_m: 0.4027 - val_single_class_accuracy: 0.0568 - val_single_class_recall: 0.6503

Epoch 00001: val_single_class_accuracy improved from -inf to 0.05678, saving model to D:\Work\projects\DL for MS\models\abd_CNN_model.001.h5
Epoch 2/200
65244/65244 [==============================] - 20s 300us/step - loss: 1.3126 - recall_m: 0.3939 - precision_m: 0.3939 - f1_m: 0.3939 - single_class_accuracy: 0.0562 - single_class_recall: 0.6621 - val_loss: 0.6934 - val_recall_m: 0.4241 - val_precision_m: 0.4241 - val_f1_m: 0.4241 - val_single_class_accuracy: 0.0583 - val_single_class_recall: 0.6430

Epoch 00002: val_single_class_accuracy improved from 0.05678 to 0.05828, saving

KeyboardInterrupt: 

In [1]:
plt.plot(model.history.history['val_single_class_accuracy'], label='true positive')
plt.plot(model.history.history['val_recall_m'], label='val_recall')

plt.ylabel('')
plt.xlabel('Epochs')
plt.legend()
# ax2 = plt.gca().twinx()
# ax2.plot(model.history.history['lr'], color='r')
# ax2.set_ylabel('lr',color='r')

_ = plt.legend()

NameError: name 'plt' is not defined

In [12]:
# Load model trained on ABD dataset

dependencies = {
    'recall_m': recall_m,'precision_m':precision_m,'f1_m':f1_m,'single_class_accuracy':single_class_accuracy,
    'single_class_recall':single_class_recall
}
model = keras.models.load_model('models/abd_CNN_model.001.h5',custom_objects=dependencies)

In [13]:
y_pred = model.predict_classes(x_test)
classification_report(y_test.argmax(axis=1), y_pred, output_dict=True)

{'0': {'precision': 0.9529284164859002,
  'recall': 0.33177252473378144,
  'f1-score': 0.49218531174724106,
  'support': 26482},
 '1': {'precision': 0.05581047913776545,
  'recall': 0.7067567567567568,
  'f1-score': 0.10345168628226684,
  'support': 1480},
 'accuracy': 0.3516200557900007,
 'macro avg': {'precision': 0.5043694478118328,
  'recall': 0.5192646407452691,
  'f1-score': 0.29781849901475393,
  'support': 27962},
 'weighted avg': {'precision': 0.9054448835742616,
  'recall': 0.3516200557900007,
  'f1-score': 0.47161003938874874,
  'support': 27962}}

In [ ]:
# # LOADING second dataset
# df_abd=pd.read_pickle('Orbitrap_aplysia/buc.pkl')
# psm_ID=list(pd.read_csv('Aplysia_ganglia/Buccal/DB search psm.csv')['Scan'])
# y=np.zeros(df_abd.shape[0])

# for i in range(0,df_abd.shape[0]):
#     if df_abd.index[i] in psm_ID:
#         y[i]=1

# X = df_abd.drop('RT',axis=1).fillna(0).values
# y =y.astype(int)[X.sum(axis=1)!=0]
# X = X[X.sum(axis=1)!=0]
# X = X/X.max(axis=1).reshape(X.shape[0],1)

# del df_abd
# X = X.reshape((X.shape[0],X.shape[1]))
# x_train, x_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=1)
# del X

In [20]:
# rdlr = ReduceLROnPlateau( min_lr=1e-4, monitor='val_single_class_accuracy', verbose=1,factor=0.8,patience=20)
# save_dir = os.path.join(os.getcwd(), 'models')
# model_name = 'buc_CNN_model.{epoch:03d}.h5' 
# if not os.path.isdir(save_dir):
#     os.makedirs(save_dir)
# filepath = os.path.join(save_dir, model_name)

# # Prepare callbacks for model saving and for learning rate adjustment.
# checkpoint = ModelCheckpoint(filepath=filepath,
#                              monitor='val_single_class_accuracy',
#                              verbose=1,
#                              save_best_only=True)

# model.fit(x_train, y_train, epochs=100, batch_size=128, validation_data=(x_test, y_test), callbacks=[rdlr,checkpoint],
#          class_weight={0:1,1:scale_weight})

Train on 32677 samples, validate on 14005 samples
Epoch 1/100
32677/32677 [==============================] - 26s 804us/step - loss: 1.3185 - recall_m: 0.9561 - precision_m: 0.9561 - f1_m: 0.9561 - single_class_accuracy: 0.1658 - val_loss: 0.6697 - val_recall_m: 0.9508 - val_precision_m: 0.9508 - val_f1_m: 0.9508 - val_single_class_accuracy: 0.3039

Epoch 00001: val_single_class_accuracy improved from -inf to 0.30391, saving model to D:\Work\projects\DL for MS\models\buc_CNN_model.001.h5
Epoch 2/100
32677/32677 [==============================] - 18s 550us/step - loss: 1.3170 - recall_m: 0.9560 - precision_m: 0.9560 - f1_m: 0.9560 - single_class_accuracy: 0.2288 - val_loss: 0.6677 - val_recall_m: 0.9514 - val_precision_m: 0.9514 - val_f1_m: 0.9514 - val_single_class_accuracy: 0.3097

Epoch 00002: val_single_class_accuracy improved from 0.30391 to 0.30970, saving model to D:\Work\projects\DL for MS\models\buc_CNN_model.002.h5
Epoch 3/100
32677/32677 [==============================] - 18s 

KeyboardInterrupt: 

In [9]:
#data = data[:,1:].reshape((data.shape[0],100,120,1))
#x_train, x_test, y_train, y_test = train_test_split(data, label, test_size=0.3, random_state=1)
del data

In [13]:
x_train.shape

(65244, 100, 120, 1)

In [89]:
import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, Reshape, MaxPooling2D#, MaxPooling1D
from keras.callbacks import ReduceLROnPlateau
from keras.layers.noise import GaussianNoise
#Hyperparameters for the network


input_dim = x_train.shape[1:]

#The model
def make_model():
    model = Sequential()
    #convolutional layer with rectified linear unit activation
    model.add(Conv2D(8, kernel_size=(16, 3),
                     activation='relu',
                     input_shape=input_shape))
    #32 convolution filters used each of size 3x3
    #again
    model.add(Conv2D(8, (1, 3), activation='relu'))
    model.add(Conv2D(8, (1, 3), activation='relu'))
    #64 convolution filters used each of size 3x3
    #choose the best features via pooling
    model.add(MaxPooling2D(pool_size=(2, 2)))
    #randomly turn neurons on and off to improve convergence
    model.add(Dropout(0.25))
    #flatten since too many dimensions, we only want a classification output
    model.add(Flatten())
    #fully connected to get all relevant data
    model.add(Dense(512, activation='relu'))
    #one more dropout for convergence' sake :) 
    model.add(Dropout(0.5))
    #output a softmax to squash the matrix into output probabilities
    model.add(Dense(num_classes, activation='softmax'))

    model.compile(loss='binary_crossentropy', optimizer=keras.optimizers.Adadelta(lr=1e-3),
                  metrics=[recall_m,precision_m,f1_m,single_class_accuracy,single_class_recall])#, metrics=['mean_absolute_error'])

    return model

In [90]:
model = make_model()
print(model.summary())

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_67 (Conv2D)           (None, 85, 118, 8)        392       
_________________________________________________________________
conv2d_68 (Conv2D)           (None, 85, 116, 8)        200       
_________________________________________________________________
conv2d_69 (Conv2D)           (None, 85, 114, 8)        200       
_________________________________________________________________
max_pooling2d_25 (MaxPooling (None, 42, 57, 8)         0         
_________________________________________________________________
dropout_49 (Dropout)         (None, 42, 57, 8)         0         
_________________________________________________________________
flatten_25 (Flatten)         (None, 19152)             0         
_________________________________________________________________
dense_47 (Dense)             (None, 512)               9806336   
__________

In [91]:
rdlr = ReduceLROnPlateau( min_lr=1e-6, monitor='val_single_class_accuracy', verbose=1,factor=0.5,patience=20)
save_dir = os.path.join(os.getcwd(), 'models')
model_name = 'abd_buc_CNN2D_model.{epoch:03d}.h5' 
if not os.path.isdir(save_dir):
    os.makedirs(save_dir)
filepath = os.path.join(save_dir, model_name)

# Prepare callbacks for model saving and for learning rate adjustment.
checkpoint = ModelCheckpoint(filepath=filepath,
                             monitor='val_single_class_accuracy',
                             verbose=1,
                             save_best_only=True)

model.fit(x_train, y_train, epochs=200, batch_size=128, validation_data=(x_test, y_test), callbacks=[rdlr,checkpoint],
         class_weight={0:1,1:scale_weight})

Train on 65244 samples, validate on 27962 samples
Epoch 1/200
65244/65244 [==============================] - 21s 327us/step - loss: 1.3129 - recall_m: 0.4690 - precision_m: 0.4690 - f1_m: 0.4690 - single_class_accuracy: 0.0540 - single_class_recall: 0.5389 - val_loss: 0.6936 - val_recall_m: 0.4779 - val_precision_m: 0.4779 - val_f1_m: 0.4779 - val_single_class_accuracy: 0.0620 - val_single_class_recall: 0.6181

Epoch 00001: val_single_class_accuracy improved from -inf to 0.06198, saving model to D:\Work\projects\DL for MS\models\abd_buc_CNN2D_model.001.h5
Epoch 2/200
65244/65244 [==============================] - 19s 295us/step - loss: 1.3121 - recall_m: 0.5247 - precision_m: 0.5247 - f1_m: 0.5247 - single_class_accuracy: 0.0613 - single_class_recall: 0.5570 - val_loss: 0.6929 - val_recall_m: 0.5737 - val_precision_m: 0.5737 - val_f1_m: 0.5737 - val_single_class_accuracy: 0.0733 - val_single_class_recall: 0.5966

Epoch 00002: val_single_class_accuracy improved from 0.06198 to 0.07331, 

KeyboardInterrupt: 